In [ ]:
#Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

# Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 3001
quant_maxima_itens = 10
variavel_controle_maximo_LOOP = 1100 #Valor maximo que da nota inicial dentro do loop de geracao de NF's

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring, pool_size=10, max_overflow=30)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2


while numero_inicial_nota < variavel_controle_maximo_LOOP:
    try:
        
        incremento = random.randint(33, 87)
        numero_maximo_nota = numero_inicial_nota + incremento
        df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)
        lista_prod_DB = df_prod['NOME'].to_list()
        df_notas_fiscais = []
        df_notas_fiscais_2 = []
        for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
            data_nota_ini = fake.date_time_between(start_date=datetime(2024,1,1),end_date=datetime(2024,1,20))
            hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
            minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
            segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
            data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
            df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
            df_notas_fiscais.append(df_nota_fiscal)
            df_notas_fiscais_2.append(df_2)
        df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
        df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)
        df_ITEMNOTA = pd.DataFrame({
                        'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                        'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                        'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                        'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                        'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                        'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas['VERSAO'],
        })
        df_NOTAFISCAL = pd.DataFrame({
                        'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                        'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                        'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                        'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                        'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                        'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                        'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas2['VERSAO'],
        })
        df_VENDA = pd.DataFrame({
                        'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                        'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                        'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas['ID_LOJA'],
                        'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                        'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                        'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas['VERSAO'],
        })
        caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
        path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

        # Caminhos das pastas
        pastas = {
            'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
            'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
            'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
        }
        # Dicionário com os DataFrames e suas pastas correspondentes
        dataframes = {
            'ITEMNOTA': df_ITEMNOTA,
            'NOTAFISCAL': df_NOTAFISCAL,
            'VENDA': df_VENDA
        }
        # Salvar cada DataFrame em um arquivo CSV na pasta correspondente
        for nome_df, df in dataframes.items():
            try:
                id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
                # Nome do arquivo
                nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"

                # Caminho completo do arquivo
                caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)

                # Salvar o DataFrame em um arquivo CSV
                df.to_excel(caminho_completo, index=False)
                #print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
            except Exception as e:
                print(f"Erro ao salvar o DataFrame {nome_df}: {e}")
        print(f"Arquivo vendas de {numero_inicial_nota} ate {numero_maximo_nota} salvos com sucesso.")
        numero_inicial_nota = numero_maximo_nota + 1
    except Exception as e:
       print(f"Erro ao gerar os arquivos: {e}") 

In [20]:
#(Otimizado) Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime
from sqlalchemy import create_engine, text
from faker import Faker

# Configurações iniciais
NUMERO_INICIAL_NOTA = 1200
VARIAVEL_CONTROLE_MAXIMO_LOOP = 1350
QUANT_MAXIMA_ITENS = 10

fake = Faker()

# Configurações do banco de dados
SERVER = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
BANCO_DADOS = "MERCADO_OLTP"
DRIVER = "ODBC Driver 17 for SQL Server"
CONN_STRING = f"mssql+pyodbc://{SERVER}/{BANCO_DADOS}?driver={DRIVER}&Trusted_Connection=yes"
engine = create_engine(CONN_STRING, pool_size=10, max_overflow=30)

# Função para testar a conexão
def testar_conexao():
    try:
        with engine.connect() as conexao:
            conexao.execute(text("SELECT 1"))
            print("Conexão com o banco de dados estabelecida com sucesso.")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False

# Função para gerar uma nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, data_nota):
    quant_itens = random.randint(1, QUANT_MAXIMA_ITENS)
    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, obter_total("CLIENTE", "IDCLIENTE"))
    id_loja = random.randint(1, obter_total("LOJA", "IDLOJA"))
    id_cfop = random.randint(1, 6)
    chave_acesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    versao = 1

    itens = []
    for _ in range(quant_itens):
        produto = random.choice(lista_produtos)
        preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
        id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
        quantidade = random.randint(1, 10)
        total_item = round(quantidade * preco_unit, 2)
        itens.append((data_nota, numero_nota, produto, quantidade, preco_unit, total_item, 
                      id_forma_pgmt, id_cliente, id_loja, id_cfop, chave_acesso, data_criacao, 
                      "", versao, id_produto))

    total_nota = round(sum(item[5] for item in itens), 2)
    itens = [item + (total_nota,) for item in itens]

    df_nota_fiscal = pd.DataFrame(
        itens, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", 
                        "TOTAL_ITEM", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                        "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO", "IDPRODUTO", "TOTAL_NOTA"]
    )

    df_nota_fiscal_2 = pd.DataFrame(
        [(data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chave_acesso, data_criacao, 
          "", versao, total_nota)],
        columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                 "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO", "TOTAL_NOTA"]
    )

    return df_nota_fiscal, df_nota_fiscal_2

# Função para obter o total de registros em uma tabela
def obter_total(tabela, coluna):
    with engine.connect() as conexao:
        return conexao.execute(text(f"SELECT COUNT({coluna}) FROM {tabela}")).scalar()

# Função para salvar DataFrames em arquivos Excel
def salvar_dataframes(dataframes, pastas, numero_inicial_nota, numero_maximo_nota):
    id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
    for nome_df, df in dataframes.items():
        try:
            nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"
            caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)
            df.to_excel(caminho_completo, index=False)
        except Exception as e:
            print(f"Erro ao salvar o DataFrame {nome_df}: {e}")

# Caminhos das pastas
PASTAS = {
    'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
    'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
    'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
}

# Execução principal
if testar_conexao():
    numero_inicial_nota = NUMERO_INICIAL_NOTA
    while numero_inicial_nota <= VARIAVEL_CONTROLE_MAXIMO_LOOP:
        try:
            incremento = random.randint(50, 99)
            numero_maximo_nota = numero_inicial_nota + incremento
            if numero_maximo_nota <= VARIAVEL_CONTROLE_MAXIMO_LOOP:
                df_prod = pd.read_sql(text("SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO"), engine)
                lista_produtos = df_prod['NOME'].to_list()

                df_notas_fiscais = []
                df_notas_fiscais_2 = []
                for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
                    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
                    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
                    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
                    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
                    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
                    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_produtos, data_nota)
                    df_notas_fiscais.append(df_nota_fiscal)
                    df_notas_fiscais_2.append(df_2)

                df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
                df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

                df_ITEMNOTA = df_todas_notas[['Numero_Nota', 'IDPRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_ITEMNOTA.columns = ['ID_NOTAFISCAL', 'ID_PRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_NOTAFISCAL = df_todas_notas2[['DATA_NOTAFISCAL', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_NOTAFISCAL.columns = ['DATA', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_VENDA = df_todas_notas[['DATA_NOTAFISCAL', 'Numero_Nota', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_NOTA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_VENDA.columns = ['DATA', 'ID_NOTAFISCAL', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_VENDA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                dataframes = {
                    'ITEMNOTA': df_ITEMNOTA,
                    'NOTAFISCAL': df_NOTAFISCAL,
                    'VENDA': df_VENDA
                }

                salvar_dataframes(dataframes, PASTAS, numero_inicial_nota, numero_maximo_nota)
                print(f"Arquivos de vendas de {numero_inicial_nota} até {numero_maximo_nota} salvos com sucesso.")
                numero_inicial_nota = numero_maximo_nota + 1
            else:
                numero_maximo_nota = VARIAVEL_CONTROLE_MAXIMO_LOOP - 1
                df_prod = pd.read_sql(text("SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO"), engine)
                lista_produtos = df_prod['NOME'].to_list()

                df_notas_fiscais = []
                df_notas_fiscais_2 = []
                for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
                    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
                    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
                    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
                    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
                    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
                    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_produtos, data_nota)
                    df_notas_fiscais.append(df_nota_fiscal)
                    df_notas_fiscais_2.append(df_2)

                df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
                df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

                df_ITEMNOTA = df_todas_notas[['Numero_Nota', 'IDPRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_ITEMNOTA.columns = ['ID_NOTAFISCAL', 'ID_PRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_NOTAFISCAL = df_todas_notas2[['DATA_NOTAFISCAL', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_NOTAFISCAL.columns = ['DATA', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                df_VENDA = df_todas_notas[['DATA_NOTAFISCAL', 'Numero_Nota', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_NOTA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
                df_VENDA.columns = ['DATA', 'ID_NOTAFISCAL', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_VENDA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

                dataframes = {
                    'ITEMNOTA': df_ITEMNOTA,
                    'NOTAFISCAL': df_NOTAFISCAL,
                    'VENDA': df_VENDA
                }

                salvar_dataframes(dataframes, PASTAS, numero_inicial_nota, numero_maximo_nota)
                print(f"Arquivos de vendas de {numero_inicial_nota} até {numero_maximo_nota} salvos com sucesso.")
                numero_inicial_nota = numero_maximo_nota + 5
        except Exception as e:
            print(f"Erro ao gerar os arquivos: {e}")

Conexão com o banco de dados estabelecida com sucesso.
Arquivos de vendas de 1200 até 1284 salvos com sucesso.
Arquivos de vendas de 1285 até 1340 salvos com sucesso.
Arquivos de vendas de 1341 até 1349 salvos com sucesso.


In [22]:
#Script para enviar arquivos da pasta Vendas par o Sql server

import random
import pandas as pd
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError
import os

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

try:
    with engine.connect() as conn:
            conn.execute(text("ALTER TABLE ITEMNOTA DROP CONSTRAINT FK_ITEMNOTA_NOTAFISCAL"))
            conn.commit()
            print(f"FK atualizada com sucesso!")
except Exception as e:
    print(f"Erro ao executar: {e}")

def ler_arquivos_xlsx(caminho_pasta):
    dataframes = []
    for arquivo in os.listdir(caminho_pasta):
        if arquivo.endswith(".xlsx"):
            try:
                caminho_completo = os.path.join(caminho_pasta, arquivo)
                df = pd.read_excel(caminho_completo)
                dataframes.append(df)
                print(f"Arquivo {arquivo} lido com sucesso.")
            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {e}")
    
    #print(f"Arquivos de {caminho_pasta} lidos com sucesso.")
    return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

#Caminhos das pastas
caminho_itemnota = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\ITEMNOTA"
caminho_notafiscal = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\NOTAFISCAL"
caminho_venda = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\VENDA"

#Lê os arquivos e cria os DataFrames
df_ITEMNOTA = ler_arquivos_xlsx(caminho_itemnota)
df_NOTAFISCAL = ler_arquivos_xlsx(caminho_notafiscal)
df_VENDA = ler_arquivos_xlsx(caminho_venda)

print("df_ITEMNOTA")
display(df_ITEMNOTA)
print("df_NOTAFISCAL")
display(df_NOTAFISCAL)
print("df_VENDA")
display(df_VENDA)

#Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Verifica se a coluna é do tipo datetime
    if np.issubdtype(col.dtype, np.datetime64):
        # Se for datetime, retorna a coluna original sem modificações
        return col
    
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip() #Converte a coluna para string, remove espaços extras e aplica strip
    
    #Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif col.name in ["DATA_CRIACAO", "DATA_ATLZ"]:  # Se for uma coluna de data, Converte valores inválidos (como 'None') para NaT (Not a Time)
        return pd.to_datetime(col_stripped, format='%Y-%m-%d', errors='coerce').dt.date
    else:  # Se for string ou outro tipo
        return col_stripped

tabela = "ITEMNOTA"          #NOME DA TABELA A SEREM ADICIONADOS OS DADOS
coluna = "ID_NOTAFISCAL"       #NOME DA COLUNA DA TABELA QUE O SCRCIPT FARA A VERIFICACAO SE O DADO JA EXISTE NO BANCO DE DADOS
tabela2 = "NOTAFISCAL" 
coluna2 = "CHAVE_ACESSO"    
tabela3 = "VENDA" 
coluna3 = "ID_NOTAFISCAL"

with engine.connect() as conn:
    # Consulta os valores existentes na coluna "NOME" da tabela "PRODUTO"
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()

    query2 = text(f"SELECT {coluna2} FROM {tabela2}")
    lista_valores_existentes2 = pd.read_sql(query2, conn)[coluna2].tolist()

    query3 = text(f"SELECT {coluna3} FROM {tabela3}")
    lista_valores_existentes3 = pd.read_sql(query3, conn)[coluna3].tolist()

    # Filtra os valores que não existem na tabela
    df_ITEMNOTA_filtrado =   df_ITEMNOTA[~df_ITEMNOTA["ID_NOTAFISCAL"].isin(lista_valores_existentes)]
    df_NOTAFISCAL_filtrado = df_NOTAFISCAL[~df_NOTAFISCAL["CHAVE_ACESSO"].isin(lista_valores_existentes2)]
    df_VENDA_filtrado =      df_VENDA[~df_VENDA["ID_NOTAFISCAL"].isin(lista_valores_existentes3)]

    # Se houver valores para adicionar
    if not df_ITEMNOTA_filtrado.empty or not df_NOTAFISCAL_filtrado.empty or not df_VENDA_filtrado.empty:
        # Limpa os espaços desnecessários
        df_ITEMNOTA_filtrado = df_ITEMNOTA_filtrado.apply(strip_all_types)
        df_NOTAFISCAL_filtrado = df_NOTAFISCAL_filtrado.apply(strip_all_types)
        df_VENDA_filtrado = df_VENDA_filtrado.apply(strip_all_types)

        # Define os tipos de dados para a tabela PRODUTO
        tipo_dados_ITEMNOTA = {
                'ID_NOTAFISCAL':  INTEGER,
                'ID_PRODUTO':     INTEGER,
                'QUANTIDADE':     INTEGER,
                'PRECO_UNIT':     NUMERIC(18,2),
                'TOTAL_ITEM':     NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
        }

        tipo_dados_NOTAFISCAL = {
                'DATA':           DATE,
                'TOTAL_NOTA':     NUMERIC(18,2),
                'ID_FORMA_PGMT':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_CFOP':        INTEGER,
                'CHAVE_ACESSO':   VARCHAR(100),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
        }

        tipo_dados_VENDA = {
                'DATA':           DATE,
                'ID_NOTAFISCAL':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_FORMA_PGMT':  INTEGER,
                'TOTAL_VENDA':    NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
        }

        # Exibe os dados que serão adicionados
        print(F"Notas a serem adicionas adicionadas ????")
        display(df_ITEMNOTA_filtrado)
        display(df_NOTAFISCAL_filtrado)
        display(df_VENDA_filtrado)

        # Adiciona os dados ao banco de dados
        df_ITEMNOTA_filtrado.to_sql(
            name=tabela,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_ITEMNOTA
        )

        df_NOTAFISCAL_filtrado.to_sql(
            name=tabela2,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_NOTAFISCAL
        )
        df_VENDA_filtrado.to_sql(
            name=tabela3,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_VENDA
        )

        print("Produtos inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

try:
    with engine.connect() as conn:
            conn.execute(text("ALTER TABLE ITEMNOTA ADD CONSTRAINT FK_ITEMNOTA_NOTAFISCAL FOREIGN KEY(ID_NOTAFISCAL) REFERENCES NOTAFISCAL(IDNOTAFISCAL)"))
            conn.commit()
            print(f"FK adicionada com sucesso!")
except Exception as e:
    print(f"Erro ao executar: {e}")


# Fecha a conexão com o banco de dados
engine.dispose()

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
FK atualizada com sucesso!
Arquivo arquivo_ITEMNOTA_vendas_1000ate_1056.xlsx lido com sucesso.
Arquivo arquivo_ITEMNOTA_vendas_1057ate_1118.xlsx lido com sucesso.
Arquivo arquivo_ITEMNOTA_vendas_1119ate_1195.xlsx lido com sucesso.
Arquivo arquivo_ITEMNOTA_vendas_1196ate_1199.xlsx lido com sucesso.
Arquivo arquivo_ITEMNOTA_vendas_1200ate_1284.xlsx lido com sucesso.
Arquivo arquivo_ITEMNOTA_vendas_1285ate_1340.xlsx lido com sucesso.
Arquivo arquivo_ITEMNOTA_vendas_1341ate_1349.xlsx lido com sucesso.
Arquivo arquivo_NOTAFISCAL_vendas_1000ate_1056.xlsx lido com sucesso.
Arquivo arquivo_NOTAFISCAL_vendas_1057ate_1118.xlsx lido com sucesso.
Arquivo arquivo_NOTAFISCAL_vendas_1119ate_1195.xlsx lido com sucesso.
Arquivo arquivo_NOTAFISCAL_vendas_1196ate_1199.xlsx lido com sucesso.
Arquivo arquivo_NOTAFISCAL_vendas_1200ate_1284.xlsx lido com sucesso.
Arquivo arquivo_NOTAFISCAL_vendas_1285

,ID_NOTAFISCAL,ID_PRODUTO,QUANTIDADE,PRECO_UNIT,TOTAL_ITEM,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,1000,34266,6,1.79,10.74,2024-01-13,NaN,1
1,1001,46553,2,8.47,16.94,2024-01-18,NaN,1
2,1001,70377,7,16.71,116.97,2024-01-18,NaN,1
3,1002,21528,7,12.09,84.63,2024-01-06,NaN,1
4,1002,220,1,8.86,8.86,2024-01-06,NaN,1
...,...,...,...,...,...,...,...,...
1912,1349,49921,6,15.70,94.20,2024-01-04,NaN,1
1913,1349,89601,1,41.12,41.12,2024-01-04,NaN,1
1914,1349,90307,9,8.32,74.88,2024-01-04,NaN,1
1915,1349,24727,2,7.51,15.02,2024-01-04,NaN,1


df_NOTAFISCAL


,DATA,TOTAL_NOTA,ID_FORMA_PGMT,ID_CLIENTE,ID_LOJA,ID_CFOP,CHAVE_ACESSO,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,2024-01-13 13:56:30,10.74,4,196078,145,2,43484859865885355527153099184907879752144346,2024-01-13,NaN,1
1,2024-01-18 07:49:50,133.91,8,620308,12,1,84185608286594594038291408158246953390200609,2024-01-18,NaN,1
2,2024-01-06 18:20:39,287.93,2,686406,17,3,79765341340628273151345749186650726213656318,2024-01-06,NaN,1
3,2024-01-15 09:10:05,337.77,1,792224,97,3,59537064417723629799155787796441751659688988,2024-01-15,NaN,1
4,2024-01-05 09:12:07,260.14,2,362228,135,1,13735614848455277933047229028468658905650022,2024-01-05,NaN,1
...,...,...,...,...,...,...,...,...,...,...
345,2024-01-19 14:54:41,1535.75,2,897454,23,4,62611543366438567385349933476369022212179316,2024-01-19,NaN,1
346,2024-01-18 08:33:21,385.31,1,153043,18,2,48106182648374100109121894883135597873398035,2024-01-18,NaN,1
347,2024-01-05 14:22:01,31.28,3,407100,84,1,12644959734030202836042095759958538929520339,2024-01-05,NaN,1
348,2024-01-19 14:16:38,772.60,7,536075,83,2,54226608183501988916685429637823510394088001,2024-01-19,NaN,1


df_VENDA


...

Notas a serem adicionas adicionadas ????


,ID_NOTAFISCAL,ID_PRODUTO,QUANTIDADE,PRECO_UNIT,TOTAL_ITEM,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,1000,34266,6,1.79,10.74,2024-01-13,NaN,1
1,1001,46553,2,8.47,16.94,2024-01-18,NaN,1
2,1001,70377,7,16.71,116.97,2024-01-18,NaN,1
3,1002,21528,7,12.09,84.63,2024-01-06,NaN,1
4,1002,220,1,8.86,8.86,2024-01-06,NaN,1
...,...,...,...,...,...,...,...,...
1912,1349,49921,6,15.70,94.20,2024-01-04,NaN,1
1913,1349,89601,1,41.12,41.12,2024-01-04,NaN,1
1914,1349,90307,9,8.32,74.88,2024-01-04,NaN,1
1915,1349,24727,2,7.51,15.02,2024-01-04,NaN,1


,DATA,TOTAL_NOTA,ID_FORMA_PGMT,ID_CLIENTE,ID_LOJA,ID_CFOP,CHAVE_ACESSO,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,2024-01-13 13:56:30,10.74,4,196078,145,2,43484859865885355527153099184907879752144346,2024-01-13,NaN,1
1,2024-01-18 07:49:50,133.91,8,620308,12,1,84185608286594594038291408158246953390200609,2024-01-18,NaN,1
2,2024-01-06 18:20:39,287.93,2,686406,17,3,79765341340628273151345749186650726213656318,2024-01-06,NaN,1
3,2024-01-15 09:10:05,337.77,1,792224,97,3,59537064417723629799155787796441751659688988,2024-01-15,NaN,1
4,2024-01-05 09:12:07,260.14,2,362228,135,1,13735614848455277933047229028468658905650022,2024-01-05,NaN,1
...,...,...,...,...,...,...,...,...,...,...
345,2024-01-19 14:54:41,1535.75,2,897454,23,4,62611543366438567385349933476369022212179316,2024-01-19,NaN,1
346,2024-01-18 08:33:21,385.31,1,153043,18,2,48106182648374100109121894883135597873398035,2024-01-18,NaN,1
347,2024-01-05 14:22:01,31.28,3,407100,84,1,12644959734030202836042095759958538929520339,2024-01-05,NaN,1
348,2024-01-19 14:16:38,772.60,7,536075,83,2,54226608183501988916685429637823510394088001,2024-01-19,NaN,1


,DATA,ID_NOTAFISCAL,ID_CLIENTE,ID_LOJA,ID_FORMA_PGMT,TOTAL_VENDA,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,2024-01-13 13:56:30,1000,196078,145,4,10.74,2024-01-13,NaN,1
1,2024-01-18 07:49:50,1001,620308,12,8,133.91,2024-01-18,NaN,1
2,2024-01-18 07:49:50,1001,620308,12,8,133.91,2024-01-18,NaN,1
3,2024-01-06 18:20:39,1002,686406,17,2,287.93,2024-01-06,NaN,1
4,2024-01-06 18:20:39,1002,686406,17,2,287.93,2024-01-06,NaN,1
...,...,...,...,...,...,...,...,...,...
1912,2024-01-04 11:08:24,1349,581896,125,2,667.05,2024-01-04,NaN,1
1913,2024-01-04 11:08:24,1349,581896,125,2,667.05,2024-01-04,NaN,1
1914,2024-01-04 11:08:24,1349,581896,125,2,667.05,2024-01-04,NaN,1
1915,2024-01-04 11:08:24,1349,581896,125,2,667.05,2024-01-04,NaN,1


Produtos inseridos com sucesso.
FK adicionada com sucesso!


In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError

# Quantidade de notas fiscais
numero_inicial_nota = 1104
numero_maximo_nota = 1145
quant_maxima_itens = 10

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")


# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)

#lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

# Caminhos das pastas
pastas = {
    'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
    'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
    'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
}
# Dicionário com os DataFrames e suas pastas correspondentes
dataframes = {
    'ITEMNOTA': df_ITEMNOTA,
    'NOTAFISCAL': df_NOTAFISCAL,
    'VENDA': df_VENDA
}

# Salvar cada DataFrame em um arquivo CSV na pasta correspondente
for nome_df, df in dataframes.items():
    try:
        id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
        # Nome do arquivo
        nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"
        
        # Caminho completo do arquivo
        caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)
        
        # Salvar o DataFrame em um arquivo CSV
        df.to_excel(caminho_completo, index=False)
        print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
    except Exception as e:
        print(f"Erro ao salvar o DataFrame {nome_df}: {e}")

In [ ]:
#Script que gera NF, ITEMNF e VENDAS em formato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

# Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 3001
quant_maxima_itens = 100
variavel_controle_maximo_LOOP = 1300 #Valor maximo que da nota inicial dentro do loop de geracao de NF's

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2


while numero_inicial_nota < variavel_controle_maximo_LOOP:
    try:
        
        incremento = random.randint(33, 87)
        numero_maximo_nota = numero_inicial_nota + incremento
        df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)
        lista_prod_DB = df_prod['NOME'].to_list()
        df_notas_fiscais = []
        df_notas_fiscais_2 = []
        for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
            data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
            hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
            minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
            segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
            data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
            df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
            df_notas_fiscais.append(df_nota_fiscal)
            df_notas_fiscais_2.append(df_2)
        df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
        df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)
        df_ITEMNOTA = pd.DataFrame({
                    'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                    'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                    'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                    'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                    'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                    'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                    'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                    'VERSAO':         df_todas_notas['VERSAO'],
        })
        df_NOTAFISCAL = pd.DataFrame({
                        'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                        'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                        'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                        'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                        'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                        'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                        'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas2['VERSAO'],
        })
        df_VENDA = pd.DataFrame({
                        'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                        'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                        'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas['ID_LOJA'],
                        'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                        'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                        'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas['VERSAO'],
        })
        caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
        path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

        # Caminhos das pastas
        pastas = {
            'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
            'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
            'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
        }
        # Dicionário com os DataFrames e suas pastas correspondentes
        dataframes = {
            'ITEMNOTA': df_ITEMNOTA,
            'NOTAFISCAL': df_NOTAFISCAL,
            'VENDA': df_VENDA
        }
        # Salvar cada DataFrame em um arquivo CSV na pasta correspondente
        for nome_df, df in dataframes.items():
            try:
                id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
                # Nome do arquivo
                nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"

                # Caminho completo do arquivo
                caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)

                # Salvar o DataFrame em um arquivo CSV
                df.to_excel(caminho_completo, index=False)
                #print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
            except Exception as e:
                print(f"Erro ao salvar o DataFrame {nome_df}: {e}")
        print(f"Arquivo vendas de {numero_inicial_nota} ate {numero_maximo_nota} salvos com sucesso.")
        numero_inicial_nota = numero_maximo_nota + 1
    except Exception as e:
       print(f"Erro ao gerar os arquivos: {e}") 